In [2]:
import numpy as np
import random

pi = np.pi
D2R = pi/180.0
R2D = 180.0/pi

phi_max = 7 # range of phi
theta_max =15
qty = 100 # number of points


rangeX = (1.6, 2.8)  # range of point distribution 
rangeY = (-0.6, 0.6)
rangeZ = (-0.3, 0.3)

##  [0,0,0] in object coordinate

In [3]:
while True:
    x = random.uniform(*rangeX)
    y = random.uniform(*rangeY)
    z = random.uniform(*rangeZ)
    ori=np.array([[x,y,z]])
    
    ori = ori.transpose()
    
    r=np.sqrt(ori[0,:]**2+ori[1,:]**2+ori[2,:]**2)
    theta = np.arctan(ori[1,:]/ori[0,:])
    phi = np.arctan(ori[2,:]/np.sqrt(ori[0,:]**2+ori[1,:]**2))
    phid = phi*R2D
    thetad = theta*R2D
    
    if phi[0]*R2D<phi_max and phi[0]*R2D>-phi_max and theta[0]*R2D<theta_max and theta[0]*R2D>-theta_max:
        break

In [4]:
print(ori)

[[ 1.90715868]
 [-0.15670568]
 [-0.18648218]]


In [6]:
theta

array([-0.08198292])

## Other points generation

In [5]:
randPoints = []
i=0
ps = []
randPoints.append([ori[0,0],ori[1,0],ori[2,0]]) # add the origin
xs = r*np.cos(theta)
ys = r*np.sin(theta)
ps.append([xs[0],ys[0]]) # add the projection of origin

while i<qty-1:
    while True:
        x = random.uniform(*rangeX)
        y = random.uniform(*rangeY)
        z = random.uniform(*rangeZ)
        xf=np.array([[x,y,z]])
    
        xf = xf.transpose()
    
        r=np.sqrt(xf[0,:]**2+xf[1,:]**2+xf[2,:]**2)
        theta = np.arctan(xf[1,:]/xf[0,:])
        phi = np.arctan(xf[2,:]/np.sqrt(xf[0,:]**2+xf[1,:]**2))
        phid = phi*R2D
        thetad = theta*R2D
    
        if phi[0]*R2D<phi_max and phi[0]*R2D>-phi_max and theta[0]*R2D<theta_max and theta[0]*R2D>-theta_max:
            break
    randPoints.append([x,y,z]) # 3d points in camera coordinates
    i += 1
    xs = r*np.cos(theta)
    ys = r*np.sin(theta)
    ps.append([xs[0],ys[0]])  # 2d points

## transfer the points to local object coordinates

In [6]:
randPoints = np.array(randPoints)
randPoints[:,0] = randPoints[:,0]-ori[0,0]
randPoints[:,1] = randPoints[:,1]-ori[1,0]
randPoints[:,2] = randPoints[:,2]-ori[2,0]

In [7]:
print(randPoints.shape)

(100, 3)


## Rotation

In [8]:
A = np.random.rand(3,3)

# Singular Value Decomposition
U, _, Vt = np.linalg.svd(A)

# Ensure a proper rotation matrix (determinant = 1)
R_svd = np.dot(U, Vt)
if np.linalg.det(R_svd) < 0:
    U[:, -1] *= -1  # Correct for reflection if necessary
    R_svd = np.dot(U, Vt)

xf = R_svd@randPoints.transpose()
xf = xf.transpose()

In [9]:
t = ori
R = R_svd

## add noise to projected points

In [10]:
noise_std = 0.0
rd1 = np.random.normal(loc=0, scale=noise_std, size=qty)
rd2 = np.random.normal(loc=0, scale=noise_std, size=qty)
    
ys1 = np.array(ps)[:,1]+rd2
ys1 = ys1[:,np.newaxis]

xs1 = np.array(ps)[:,0]+rd1
xs1 = xs1[:,np.newaxis]
    
#ps = np.array(ps)
ps = np.stack([xs1[:,0],ys1[:,0]])
